A mixture of $N =10$ random Gaussians centered in $[-1,1]^n$ 
$$
f_{\mu,\sigma}(x, y) = \sum_{i=1}^{N} \frac{1}{\sigma_i\sqrt{2\pi}}\exp\left(-\frac{(x-\mu_i)^2}{2\sigma_i^2}\right)
$$
 is defined over the square $[-1, 1]^2$.

In [9]:
using Globtim
include("../../../src/lib_func.jl") # Include the library of functions

# Constants and Parameters
d = 1 # Initial Degree 
const n, a, b = 4, 1, 1 
const scale_factor = a / b       # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .5 , 8 / 10  # Sampling parameters
const tol_l2 = 5e-4             # Define the tolerance for the L2-norm

N = 10
params = init_gaussian_params(n, N, .2)
# Create a closure that captures params
rand_gaussian_closure = (x) -> rand_gaussian(x, params)
f = rand_gaussian_closure; # Function to be optimized


In [10]:
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, n, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        global d += 1
    end
end;

current L2-norm: 0.016612268352545853
Number of samples: 2
current L2-norm: 0.015422245239644845
Number of samples: 2
current L2-norm: 0.005386411187192428
Number of samples: 3
current L2-norm: 0.005055313486721504
Number of samples: 3
current L2-norm: 0.001367100434161546
Number of samples: 4
current L2-norm: 0.0016249258209419257
Number of samples: 5
current L2-norm: 0.0014821438508829223
Number of samples: 5
current L2-norm: 0.0006132011896383424
Number of samples: 6
current L2-norm: 0.0006255439894406815
Number of samples: 7
attained the desired L2-norm: 0.00041145251341097745


In [11]:
using DynamicPolynomials, HomotopyContinuation, ProgressLogging, DataFrames
@polyvar(x[1:n]) # Define polynomial ring 
ap = main_nd(n, d, poly_approx.coeffs)
# Expand the polynomial approximant to the standard monomial basis in the Lexicographic order w.r.t x. 
PolynomialApproximant = sum(Float64.(ap) .* MonomialVector(x, 0:d)) # Convert coefficients to Float64 for homotopy continuation
grad = differentiate.(PolynomialApproximant, x)
sys = System(grad)
println("The system is of degree:", d-1)

The system is of degree:9


In [12]:
Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false);

Computing mixed cells... 2    Time: 0:00:00
Computing mixed cells... 151    Time: 0:00:00
  mixed_volume:  6561


Tracking 6561 paths...   0%|                            |  ETA: 10.38 days



Tracking 6561 paths...   0%|▏                           |  ETA: 1 days, 7:10:25



Tracking 6561 paths...   0%|▏                           |  ETA: 22:40:17



Tracking 6561 paths...   0%|▏                           |  ETA: 17:48:41



Tracking 6561 paths...   1%|▏                           |  ETA: 14:40:03



Tracking 6561 paths...   1%|▏                           |  ETA: 12:47:13



Tracking 6561 paths...   1%|▎                           |  ETA: 11:20:04



Tracking 6561 paths...   1%|▎                           |  ETA: 10:23:29



Tracking 6561 paths...   1%|▎                           |  ETA: 9:35:36



Tracking 6561 paths...   1%|▎                           |  ETA: 8:35:58



Tracking 6561 paths...   1%|▎                           |  ETA: 7:55:03



Tracking 6561 paths...   1%|▎                           |  ETA: 7:20:07



Tracking 6561 paths...   1%|▎                           |  ETA: 6:55:46



Tracking

Excessive output truncated after 524308 bytes.





Tracking 6561 paths...  87%|████████████████████████▍   |  ETA: 0:01:25
  # paths tracked:                  5714
  # non-singular solutions (real):  5714 (2568)

In [26]:
function condition(point::Vector{Float64}, n::Int)::Bool
    return all(-1 < point[i] < 1 for i in 1:n)
end
# condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1
# filtered_points = filter(condition, real_pts) # Filter points using the filter function
filtered_points = filter(p -> condition(p, n), real_pts) # Filter points using the condition function

# Colllect the critical points of the approximant 
# h_x = Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
# h_y = Float64[point[2] for point in filtered_points] # Initialize the y vector
# h_z = map(p -> f([p[1], p[2]]), zip(scale_factor * h_x, scale_factor * h_y))
# df = DataFrame(x=scale_factor * h_x, y=scale_factor * h_y, z=h_z) # Create a DataFrame

2821-element Vector{Vector{Float64}}:
 [0.9387168343090977, 0.7796454913945097, 0.8952070280867294, 0.13346469598021862]
 [0.943234037423944, 0.9402857731040115, 0.1502808405364178, 0.9460489128124056]
 [-0.8816940205841061, 0.17473320577506507, -0.5721648636930791, -0.15636343906839095]
 [0.9272638084442071, -0.1709709424040544, -0.9380703315230209, -0.6926298893964523]
 [0.12720017837586572, 0.9383684865189793, -0.7396251385008101, -0.7890323359912553]
 [0.0824255354530138, -0.19311152674933604, -0.2637748674778437, 0.273928655368467]
 [0.1489465207518041, -0.7667096657698474, 0.2319259676754426, 0.4048000274926785]
 [0.9392726988058155, 0.8463386853303091, 0.9524096556030459, 0.9417877064184706]
 [0.9427224403533379, 0.5059696541521495, 0.15679098423502721, 0.7291600169190707]
 [-0.10925143735351966, -0.7699347192927464, 0.17278046102290842, 0.9560981607046101]
 ⋮
 [-0.4827157244817292, -0.9401809348082544, 0.49709237979068643, 0.8504839239525984]
 [-0.5087116095630534, 0.9397362669

In [27]:
println(length(real_pts))

Repeat solving with exact method, compare timing. 

In [ ]:
loc = "inputs.ms"
# File path of the output file
file_path_output = "outputs.ms";

using DynamicPolynomials, DataFrames
ap = main_nd(n, d, poly_approx.coeffs)
@polyvar(x[1:n]) # Define polynomial ring 
# Expand the polynomial approximant to the standard monomial basis in the Lexicographic order w.r.t x
names = [x[i].name for i in 1:length(x)]
open(loc, "w") do file
    println(file, join(names, ", "))
    println(file, 0)
end
# Define the polynomial approximant 
PolynomialApproximant = sum(ap .* MonomialVector(x, 0:d))
for i in 1:n
    partial = differentiate(PolynomialApproximant, x[i])
    partial_str = replace(string(partial), "//" => "/")
    open(loc, "a") do file
        if i < n
            println(file, string(partial_str, ","))
        else
            println(file, partial_str)
        end
    end
end

In [15]:
# Optimize the collected entries 
using Optim
for i in 1:nrow(df)
    println("Optimizing for point $i")
    x0 = [df.x[i], df.y[i]]
    res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=true))
    minimizer = Optim.minimizer(res)
    min_value = Optim.minimum(res)
    steps = res.iterations
    converged = Optim.converged(res)
    distance = norm(x0 - minimizer)
    println(summary(res))
end


UndefVarError: UndefVarError: `df` not defined

Should plot the polynomial approximant too.